In [5]:
import pandas as pd
import numpy as np
import xlsxwriter
from prophet import Prophet
from numpy import concatenate
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from datetime import datetime
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from math import sqrt

# prophet univariate
all_df = pd.read_csv("C:/users/FurkanBulut/Desktop/T1totalclean.csv", usecols=['Name', 'WS101']) # veri setini yükle
all_df.columns = ['ds', 'y']
results = []

for i in range(0, 57601, 2):
    df = all_df[i:i+144]
    m = Prophet(daily_seasonality=True, weekly_seasonality=True, yearly_seasonality=True)
    m.fit(df)
    future = m.make_future_dataframe(periods=144, freq='10 min')
    forecast = m.predict(future)
    result = forecast[['ds', 'yhat']]
    result = result.tail(2)
    results.append(result)

# prophet multivariate
train = pd.read_csv('iztech.csv', usecols=['Name', 'WS101', 'RH3', 'T3', 'P2'])
train.rename(columns={'Name': 'ds', 'RH3': 'add1', 'T3': 'add2', 'P2': 'add3', 'WS101': 'y'}, inplace=True)
results_multivariate = []

for i in range(0, 57601, 2):
    X_train = train.iloc[i:i+144]
    X_val = train.iloc[i+144:i+146]
    model = Prophet(daily_seasonality=True, weekly_seasonality=True, yearly_seasonality=True)
    model.add_regressor('add1')
    model.add_regressor('add2')
    model.add_regressor('add3')
    model.fit(X_train)
    forecast_1 = model.predict(X_val.drop(columns="y"))
    result_1 = forecast_1[['ds', 'yhat']]
    results_multivariate.append(result_1)

# Create Pandas dataframes from forecasts
df1 = pd.read_csv('iztech.csv', usecols=['Name', 'T3'])
df1.columns = ['ds', 'y']
df1 = df1[144:57601]
df1 = pd.DataFrame(df1)
df2 = pd.DataFrame(np.concatenate(results), columns=['ds', 'yhat'])
df3 = pd.DataFrame(np.concatenate(results_multivariate), columns=['ds', 'yhat'])
expected = df1['y']
prediction_multi = df2['yhat']
prediction_uni = df3['yhat']

MSE_multi = mean_squared_error(expected, prediction_multi)
R2_multi = r2_score(expected, prediction_multi)
RMSE_multi = sqrt(mean_squared_error(expected, prediction_multi))
MAE_multi = mean_absolute_error(expected, prediction_multi)

print("MSE Multi:", MSE_multi)
print("R2 Multi:", R2_multi)
print("RMSE Multi:", RMSE_multi)
print("MAE Multi:", MAE_multi)


Importing plotly failed. Interactive plots will not work.


FileNotFoundError: [Errno 2] No such file or directory: 'iztech.csv'

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
import numpy as np
import pandas as pd
from itertools import product
from sklearn.metrics import *
from math import sqrt
import warnings

warnings.filterwarnings('ignore')

# Load your data
all_df = pd.read_csv('iztech.csv', usecols=['Name', 'WS101'])
forecast = []

# Loop over your data
for i in range(0, 57601, 2):
    data = all_df[i:i + 144]

    def optimize_SARIMA(endog, parameters_list, d, D, s):
        results = []
        for param in tqdm_notebook(parameters_list):
            try:
                model = SARIMAX(endog, order=(param[0], d, param[1]), seasonal_order=(param[2], D, param[3], s),
                                simple_differencing=False).fit(disp=False)
            except:
                continue
            aic = model.aic
            results.append([param, aic])

        result_df = pd.DataFrame(results)
        result_df.columns = ['(p,q)x(P,Q)', 'AIC']
        # Sort in ascending order, lower AIC is better
        result_df = result_df.sort_values(by='AIC', ascending=True).reset_index(drop=True)
        return result_df

    p = range(0, 3, 1)
    d = 1
    q = range(0, 3, 1)
    P = range(0, 3, 1)
    D = 1
    Q = range(0, 3, 1)
    s = 4
    parameters = product(p, q, P, Q)
    parameters_list = list(parameters)

    result_df = optimize_SARIMA(data['WS101'], parameters_list, 1, 1, 4)
    result_df_min = result_df[result_df.AIC == result_df.AIC.min()]
    p = result_df_min['(p,q)x(P,Q)'][0][0]
    q = result_df_min['(p,q)x(P,Q)'][0][1]
    P = result_df_min['(p,q)x(P,Q)'][0][2]
    Q = result_df_min['(p,q)x(P,Q)'][0][3]
    best_model = SARIMAX(data['WS101'], order=(p, d, q), seasonal_order=(P, D, Q, s), simple_differencing=False)
    res = best_model.fit(disp=False)
    n_forecast = 2
    predict = res.get_prediction(end=best_model.nobs + n_forecast)
    prediction = predict.predicted_mean[-n_forecast:]
    forecast.append(prediction)

df = all_df[144:57601]
df1 = pd.DataFrame(np.concatenate(forecast))
expected = df['WS101']
prediction = df1
MSE = mean_squared_error(expected, prediction)
R2 = r2_score(expected, prediction)
RMSE = sqrt(mean_squared_error(expected, prediction))
MAE = mean_absolute_error(expected, prediction)

print("MSE:", MSE)
print("R2:", R2)
print("RMSE:", RMSE)
print("MAE:", MAE)


In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from itertools import product
import warnings
from sklearn.metrics import *

warnings.filterwarnings('ignore')

%matplotlib inline

macro_data = pd.read_csv('iztech.csv', usecols=['Name', 'T3', 'RH3', 'P2', 'WS101'])
forecast = []

for i in range(0, 144, 2):
    data = macro_data[i:i+144]

    def optimize_SARIMAX(endog, exog, parameters_list, d, D, s):
        results = []
        for param in tqdm_notebook(parameters_list):
            try:
                model = SARIMAX(endog,
                                exog,
                                order=(param[0], d, param[1]),
                                seasonal_order=(param[2], D, param[3]),
                                simple_differencing=False).fit(disp=False)
            except:
                continue
            aic = model.aic
            results.append([param, aic])

        result_df = pd.DataFrame(results)
        result_df.columns = ['(p,q)x(P,Q)', 'AIC']
        result_df = result_df.sort_values(by='AIC', ascending=True).reset_index(drop=True)
        return result_df

    p = range(0, 3, 1)
    d = 1
    q = range(0, 3, 1)
    P = range(0, 3, 1)
    D = 0
    Q = range(0, 3, 1)
    s = 4
    parameters = product(p, q, P, Q)
    parameters_list = list(parameters)
    endog = macro_data['WS101'][i:i+142]
    exog = macro_data[['T3', 'RH3', 'P2']][i:i+142]
    result_df = optimize_SARIMAX(endog, exog, parameters_list, 1, 0, 4)
    result_df_min = result_df[result_df.AIC == result_df.AIC.min()]
    p = result_df_min['(p,q)x(P,Q)'][0][0]
    q = result_df_min['(p,q)x(P,Q)'][0][1]
    P = result_df_min['(p,q)x(P,Q)'][0][2]
    Q = result_df_min['(p,q)x(P,Q)'][0][3]
    best_model = SARIMAX(endog,
                        exog,
                        order=(p, d, q),
                        seasonal_order=(P, D, Q, s),
                        simple_differencing=False)
    res = best_model.fit(dis=False)
    n_forecast = 2
    predict = res.get_prediction(end=best_model.nobs + n_forecast, exog=exog.iloc[-3:])
    prediction = predict.predicted_mean[-n_forecast:]
    forecast.append(prediction)

df = macro_data['WS101']
expected = df[144:57601]
prediction = pd.DataFrame(np.concatenate(forecast))
MSE = mean_squared_error(expected, prediction)
R2 = r2_score(expected, prediction)
RMSE = sqrt(mean_squared_error(expected, prediction))
MAE = mean_absolute_error(expected, prediction)

print("MSE:", MSE)
print("R2:", R2)
print("RMSE:", RMSE)
print("MAE:", MAE)


In [ ]:
import pandas as pd
import numpy as np
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot as plt
from pandas import read_csv, DataFrame, concat
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU
import tensorflow as tf
from datetime import datetime

plt.rcParams['figure.figsize'] = (15, 5)

# Load the dataset
dataset = read_csv("iztech.csv",
                   parse_dates={'dt': ['Name']},
                   infer_datetime_format=True,
                   index_col=0,
                   na_values=['nan', '?'])
dataset.fillna(0, inplace=True)

# Drop unnecessary columns
dataset.drop(columns=['WS101Max', 'WS101Min', 'WS101Count', 'WS 76', 'WS76Max', 'WS76Min', 'WS76Std', 'WS76Count', 'WD28Max', 'WD28Min',
                      'WD28Std', 'WD28Count', 'rho10', 'rho10Max', 'rho10Std'], inplace=True)

dataset.drop(columns=['WS101Std', 'WS30', 'WS30Max', 'WS30Min', 'WS30Std', 'WS30Count', 'WD74', 'WD74Max', 'WD74Min', 'WD74Std',
                      'WD74Count', 'WD28'], inplace=True)

dataset = dataset[['T3', 'P2', 'RH3', 'WS101']]
values = dataset.values

# Ensure all data is float
values = values.astype('float32')

# Normalize input features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
scaled = pd.DataFrame(scaled)

# Create a function to prepare time series data
def create_ts_data(dataset, lookback=1, predicted_col=3):
    temp = dataset.copy()
    temp["id"] = range(1, len(temp) + 1)
    temp = temp.iloc[:-lookback, :]
    temp.set_index('id', inplace=True)
    
    predicted_value = dataset.copy()
    predicted_value = predicted_value.iloc[lookback:, predicted_col]
    predicted_value.columns = ["Predicted"]
    predicted_value = pd.DataFrame(predicted_value)
    predicted_value["id"] = range(1, len(predicted_value) + 1)
    predicted_value.set_index('id', inplace=True)
    
    final_df = pd.concat([temp, predicted_value], axis=1)
    
    return final_df

# Create time series data with a lookback of 1
reframed_df = create_ts_data(scaled, 1, 3)
reframed_df.fillna(0, inplace=True)
reframed_df.columns = ['var1(t-1)', 'var2(t-1)', 'var3(t-1)', 'var4(t-1)', 'var5(t-1)']

# Split data into train and test sets
values = reframed_df.values
training_sample = int(len(dataset) * 0.7)
train = values[:training_sample, :]
test = values[training_sample:, :]

# Split data into input and output
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]

# Reshape input to be 3D [samples, time steps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

# Build and compile the GRU model
model_gru = Sequential()
model_gru.add(GRU(75, return_sequences=True, input_shape=(train_X.shape[1], train_X.shape[2])))
model_gru.add(GRU(units=30, return_sequences=True))
model_gru.add(GRU(units=30))
model_gru.add(Dense(units=1))
model_gru.compile(loss='mae', optimizer='adam')

# Display model summary
model_gru.summary()

# Train the model
gru_history = model_gru.fit(train_X, train_y, epochs=10, validation_data=(test_X, test_y), batch_size=64, shuffle=False)

# Predictions
pred_y = model_gru.predict(test_X)
pred_y_1 = model_gru.predict(train_X)

# Plot training and validation loss
plt.plot(gru_history.history['loss'], label='GRU train', color='brown')
plt.plot(gru_history.history['val_loss'], label='GRU test', color='blue')
plt.legend()
plt.show()

# Calculate evaluation metrics
MSE = mean_squared_error(test_y, pred_y)
R2 = r2_score(test_y, pred_y)
RMSE = sqrt(mean_squared_error(test_y, pred_y))
MAE = mean_absolute_error(test_y, pred_y)

print("MSE:", MSE)
print("R2:", R2)
print("RMSE:", RMSE)
print("MAE:", MAE)

# Plot predicted vs actual test values
plt.plot(test_y, label='Actual')
plt.plot(pred_y, label='Predicted')
plt.legend()
plt.show()

# Visualization over the full data
tra = np.concatenate([train_X, test_X])
tes = np.concatenate([train_y, test_y])
fp = model_gru.predict(tra)
plt.plot(tes, label='Actual')
plt.plot(fp, label='Predicted')
plt.legend()
plt.show()

# Inverse normalizing
dataset_1 = dataset[['WS101']]
dataset_1 = dataset_1[40421:57744]
values_1 = dataset_1.values

# Ensure all data is float
values_1 = values_1.astype('float32')

# Normalize input features
scaler_1 = MinMaxScaler(feature_range=(0, 1))
scaled_1 = scaler_1.fit_transform(values_1)
scaled_1 = pd.DataFrame(scaled_1)

inversed_1 = scaler_1.inverse_transform(scaled_1)
print(inversed_1)
print(values_1)
print(scaled_1)

test_y_inv = scaler_1.inverse_transform([test_y])
test_y_inv = test_y_inv.reshape(17323, 1)

pred_y_inv = scaler_1.inverse_transform(pred_y)
pred_y_1_inv = scaler_1.inverse_transform(pred_y_1)

MSE = mean_squared_error(test_y_inv, pred_y_inv)
R2 = r2_score(test_y_inv, pred_y_inv)
RMSE = sqrt(mean_squared_error(test_y_inv, pred_y_inv))
MAE = mean_absolute_error(test_y_inv, pred_y_inv)

print("MSE (Inverse Normalized):", MSE)
print("R2 (Inverse Normalized):", R2)
print("RMSE (Inverse Normalized):", RMSE)
print("MAE (Inverse Normalized):", MAE)


In [ ]:
import pandas as pd
import numpy as np
from math import sqrt
from numpy import concatenate
import matplotlib.pyplot as plt
from pandas import read_csv, DataFrame, concat
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Load the dataset
dataset = read_csv("iztech.csv",
                   parse_dates={'dt': ['Name']},
                   infer_datetime_format=True,
                   index_col=0,
                   na_values=['nan', '?'])
dataset.fillna(0, inplace=True)
values = dataset.values
dataset.head(4)

# Drop unnecessary columns
dataset.drop(columns=['WS101Max', 'WS101Min', 'WS101Count', 'WS76', 'WS76Max', 'WS76Min', 'WS76Std', 'WS76Count', 'WD28Max', 'WD28Min', 'WD28Std', 'WD28Count', 'rho10', 'rho10Max', 'rho10Std'], inplace=True)
dataset.drop(columns=['WS101Std', 'WS30', 'WS30Max', 'WS30Min', 'WS30Std', 'WS30Count', 'WD74', 'WD74Max', 'WD74Min', 'WD74Std', 'WD74Count', 'WD28'], inplace=True)
dataset = dataset[['T3', 'P2', 'RH3', 'WS101']]
values = dataset.values

# Ensure all data is float
values = values.astype('float32')

# Normalize input features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
scaled = pd.DataFrame(scaled)

def create_ts_data(dataset, lookback=1, predicted_col=3):
    temp = dataset.copy()
    temp["id"] = range(1, len(temp) + 1)
    temp = temp.iloc[:-lookback, :]
    temp.set_index('id', inplace=True)

    predicted_value = dataset.copy()
    predicted_value = predicted_value.iloc[lookback:, predicted_col]
    predicted_value.columns = ["Predicted"]
    predicted_value = pd.DataFrame(predicted_value)
    predicted_value["id"] = range(1, len(predicted_value) + 1)
    predicted_value.set_index('id', inplace=True)

    final_df = pd.concat([temp, predicted_value], axis=1)

    return final_df

# Create time series data with a lookback of 1
reframed_df = create_ts_data(scaled, 1, 3)
reframed_df.fillna(0, inplace=True)
reframed_df.columns = ['var1(t-1)', 'var2(t-1)', 'var3(t-1)', 'var4(t-1)', 'var5(t-1)']

# Split data into train and test sets
values = reframed_df.values
training_sample = int(len(dataset) * 0.7)
train = values[:training_sample, :]
test = values[training_sample:, :]

# Split data into input and output
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]

# Reshape input to be 3D [samples, time steps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

model_lstm = Sequential()
model_lstm.add(LSTM(75, return_sequences=True, input_shape=(train_X.shape[1], train_X.shape[2])))
model_lstm.add(LSTM(units=30, return_sequences=True))
model_lstm.add(LSTM(units=30))
model_lstm.add(Dense(units=1))
model_lstm.compile(loss='mae', optimizer='adam')

model_lstm.summary()

lstm_history = model_lstm.fit(train_X, train_y, epochs=10, validation_data=(test_X, test_y), batch_size=64, shuffle=False)
pred_y = model_lstm.predict(test_X)
pred_y_1 = model_lstm.predict(train_X)

plt.plot(lstm_history.history['loss'], label='lstm train', color='brown')
plt.plot(lstm_history.history['val_loss'], label='lstm test', color='blue')
plt.legend()
plt.show()

MSE = mean_squared_error(test_y, pred_y)
R2 = r2_score(test_y, pred_y)
RMSE = sqrt(mean_squared_error(test_y, pred_y))
MAE = mean_absolute_error(test_y, pred_y)

print("MSE:", MSE)
print("R2:", R2)
print("RMSE:", RMSE)
print("MAE:", MAE)

plt.plot(test_y, label='Actual')
plt.plot(pred_y, label='Predicted')
plt.legend()
plt.show()

tra = np.concatenate([train_X, test_X])
tes = np.concatenate([train_y, test_y])
fp = model_lstm.predict(tra)
plt.plot(tes, label='Actual')
plt.plot(fp, label='Predicted')
plt.legend()
plt.show()

dataset_1 = dataset[['WS101']]
dataset_1 = dataset_1[40421:57744]
values_1 = dataset_1.values

values_1 = values_1.astype('float32')

scaler_1 = MinMaxScaler(feature_range=(0, 1))
scaled_1 = scaler_1.fit_transform(values_1)
scaled_1 = pd.DataFrame(scaled_1)

inversed_1 = scaler_1.inverse_transform(scaled_1)
print(inversed_1)
print(values_1)
print(scaled_1)

test_y_inv = scaler_1.inverse_transform([test_y])
test_y_inv = test_y_inv.reshape(17323, 1)

pred_y_inv = scaler_1.inverse_transform(pred_y)
pred_y_1_inv = scaler_1.inverse_transform(pred_y_1)

MSE = mean_squared_error(test_y_inv, pred_y_inv)
R2 = r2_score(test_y_inv, pred_y_inv)
RMSE = sqrt(mean_squared_error(test_y_inv, pred_y_inv))
MAE = mean_absolute_error(test_y_inv, pred_y_inv)

print("MSE (Inverse Normalized):", MSE)
print("R2 (Inverse Normalized):", R2)
print("RMSE (Inverse Normalized):", RMSE)
print("MAE (Inverse Normalized):", MAE)
